# Second-Order Maximum Mean Discrepancy of Multi-Dimensional Geometric Brownian Motion

## Change working directory to navigate out of folder. 

In [1]:
import os
os.getcwd()
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

## Import Packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
import math
from tqdm.auto import tqdm
from collections import defaultdict
from collections import Counter
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
import torch.cuda

from MMD.mmd import RBFKernel, SigKernel

from GenerateMMDDataset.mmd_dataset_base import save_dataset, load_dataset, save_path_params, load_path_params
from GenerateMMDDataset.multi_gbm_dataset import GenerateMultiDimensionalGBM

from StochasticModels.geometric_brownian_motion import BlackScholesExactSimulationSobolNDim

from RegressionModel.pricing_model import PricingModel
from RegressionModel.mmd_model import SecondOrderMMDApprox
from RegressionModel.regression_model import RegressionModel, RegressionNeuralNetwork, DatasetDataLoader

## Set PyTorch Device

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')
torch.backends.cudnn.benchmark = True

Device: cuda


## Neural Network Approximation

### Generate Dataset

In [ ]:
genMultiGBMData = GenerateMultiDimensionalGBM(20, device, 50, 0, 3, 0.5)
multi_gbm_mmd_dataset, multi_gbm_path_dataset, multi_gbm_sample_path_param_dict = genMultiGBMData.generate_mmd_dataset(1000, 24, 200, (1, 1), (0.2, 0.8), estimator='ub')

In [ ]:
torch.save(multi_gbm_mmd_dataset, 'Multi GBM/Data/multi_gbm_mmd_dataset_1000_second_order_samples_ub.pt')

save_path_params(multi_gbm_sample_path_param_dict, 
                 'Multi GBM/Data/multi_gbm_sample_path_param_dict_1000_second_order_samples_ub.pkl')

### Load Dataset

In [4]:
multi_gbm_mmd_dataset = torch.load('Multi GBM/Data/multi_gbm_mmd_dataset_1000_second_order_samples_ub.pt')

multi_gbm_sample_path_param_dict = load_path_params('Multi GBM/Data/multi_gbm_sample_path_param_dict_1000_second_order_samples_ub.pkl')

### Training the Model

In [9]:
M_multi_gbm = multi_gbm_sample_path_param_dict['M']
N_multi_gbm = multi_gbm_sample_path_param_dict['N']
dim = multi_gbm_sample_path_param_dict['Dim']


multi_gbm_mmd_features = []
multi_gbm_mmd_labels = []
for i in range(M_multi_gbm):
    for j in range(N_multi_gbm): 
        new_feature = [multi_gbm_sample_path_param_dict['Sigma'][i][int(k/2)] if k%2 == 0 else multi_gbm_sample_path_param_dict['Sigma'][j][int(k/2)] for k in range(2*dim)]
        new_feature += [multi_gbm_sample_path_param_dict['Correlation'][i], multi_gbm_sample_path_param_dict['Correlation'][j]]
        multi_gbm_mmd_features.append(new_feature)
        multi_gbm_mmd_labels.append(multi_gbm_mmd_dataset[i][j])

In [ ]:
multi_gbm_mmd_training_param_dict = {
    'lr' : 3e-3,
    'Epochs' : 200,
    'l2_weight' : 0.0,
    'l1_weight' : 0.0,
    'Train/Val Split' : 0.8
}


multi_gbm_mmd_dataset_loader_params = {
    'batch_size' : 256,
    'shuffle' : True,
    'num_workers' : 0
}

multi_gbm_mmd_input_dimension = 8

multi_gbm_mmd_model_param_dict = {
    'input_dimension' : multi_gbm_mmd_input_dimension,
    'intermediate_dimensions' : [30, 30, 30, 30],
    'activation_functions' : [nn.ReLU(), nn.ReLU(), nn.ReLU(), nn.ReLU()],
    'add_layer_norm' : [False, False, False, False], 
    'output_dimension' : 1,
    'output_activation_fn' : None
}

for i in range(5):
    
    multi_gbm_second_order_mmd_model = SecondOrderMMDApprox(multi_gbm_mmd_model_param_dict, 
                                                        multi_gbm_mmd_training_param_dict,
                                                        multi_gbm_mmd_dataset_loader_params, nn.MSELoss(), device,
                                                        f'Multi GBM/MMD Model/multi_gbm_mmd_scaler_{i+1}.pkl')
    
    multi_gbm_second_order_mmd_model.fit(torch.tensor(multi_gbm_mmd_features).float(),
                                     torch.tensor(multi_gbm_mmd_labels).float(),
                                     **{'filename' : f'Multi GBM/MMD Model/multi_gbm_mmd_checkpoint_{i+1}.pth.tar',
                                        'best_model_filename' : f'Multi GBM/MMD Model/multi_gbm_mmd_model_best_{i+1}.pth.tar'})

In [14]:
train_losses = []
valid_losses = []
for i in range(5):
    
    checkpoint = torch.load(f'Multi GBM/MMD Model/multi_gbm_mmd_model_best_{i+1}.pth.tar', map_location='cpu')
    train_losses.append(checkpoint['train_loss'])
    valid_losses.append(checkpoint['valid_loss'])
    
print(np.mean(train_losses))
print(np.std(train_losses))

print(np.mean(valid_losses))
print(np.std(valid_losses))

9.877856791717932e-05
4.9644229827584214e-06
0.00010450972513353918
1.4837267565938272e-06


### Test Model

In [13]:
genMultiGBMData = GenerateMultiDimensionalGBM(20, device, 50, 0, 3, 0.5)
multi_gbm_mmd_dataset, multi_gbm_path_dataset, multi_gbm_sample_path_param_dict = genMultiGBMData.generate_mmd_dataset(40, 24, 200, (1, 1), (0.2, 0.8), estimator='ub')


M_multi_gbm = multi_gbm_sample_path_param_dict['M']
N_multi_gbm = multi_gbm_sample_path_param_dict['N']
dim = multi_gbm_sample_path_param_dict['Dim']
multi_gbm_mmd_features = []
multi_gbm_mmd_labels = []
for i in range(M_multi_gbm):
    for j in range(N_multi_gbm): 
        new_feature = [multi_gbm_sample_path_param_dict['Sigma'][i][int(k/2)] if k%2 == 0 else multi_gbm_sample_path_param_dict['Sigma'][j][int(k/2)] for k in range(2*dim)]
        new_feature += [multi_gbm_sample_path_param_dict['Correlation'][i], multi_gbm_sample_path_param_dict['Correlation'][j]]
        multi_gbm_mmd_features.append(new_feature)
        multi_gbm_mmd_labels.append(multi_gbm_mmd_dataset[i][j].cpu())

Computing Initial Distances


  0%|          | 0/20 [00:00<?, ?it/s]

Finished Computing Initial Distances
----------------------------------------------------------------------------------------------------


Computing Next Batch of Distances


  0%|          | 0/20 [00:00<?, ?it/s]

Finished Computing All Distances
----------------------------------------------------------------------------------------------------




In [21]:
test_losses = []

for i in range(5):
    
    multi_gbm_second_order_mmd_model = SecondOrderMMDApprox(multi_gbm_mmd_model_param_dict, 
                                                        multi_gbm_mmd_training_param_dict,
                                                        multi_gbm_mmd_dataset_loader_params, nn.MSELoss(), device,
                                                        f'Multi GBM/MMD Model/multi_gbm_mmd_scaler_{i+1}.pkl')
    
    multi_gbm_second_order_mmd_model.load_best_model(f'Multi GBM/MMD Model/multi_gbm_mmd_model_best_{i+1}.pth.tar', 
                                                     f'Multi GBM/MMD Model/multi_gbm_mmd_scaler_{i+1}.pkl')
    
    multi_gbm_second_order_mmd_model.model.eval()
    with torch.no_grad():
        
        distances = multi_gbm_second_order_mmd_model.transform(torch.tensor(multi_gbm_mmd_features))
        test_losses.append(mean_squared_error(distances.cpu(), multi_gbm_mmd_labels))
        
print(np.mean(test_losses))
print(np.std(test_losses))

0.00219359314125947
0.00014377245652035786


### Test to Determine whether $\mathcal{NN}_{2}^{\text{MGBM}} \left( \cdot, \cdot \right)$ is a Metric

#### Load Model

In [23]:
multi_gbm_mmd_input_dimension = 8

multi_gbm_mmd_model_param_dict = {
    'input_dimension' : multi_gbm_mmd_input_dimension,
    'intermediate_dimensions' : [30, 30, 30, 30],
    'activation_functions' : [nn.ReLU(), nn.ReLU(), nn.ReLU(), nn.ReLU()], 
    'add_layer_norm' : [False, False, False, False], 
    'output_dimension' : 1,
    'output_activation_fn' : None
}

multi_gbm_second_order_mmd_model = SecondOrderMMDApprox(multi_gbm_mmd_model_param_dict, 
                                                        None,
                                                        None, None, device,
                                                        'Multi GBM/MMD Model/multi_gbm_mmd_scaler_1.pkl')

multi_gbm_second_order_mmd_model.load_best_model('Multi GBM/MMD Model/multi_gbm_mmd_model_best_1.pth.tar', 
                                                 'Multi GBM/MMD Model/multi_gbm_mmd_scaler_1.pkl')

- #### Identity of Indiscernables: $\sqrt{\mathcal{NN}_{2}^{\text{MGBM}} \left(\Theta, \Theta \right)} = 0$. 

In [28]:
mmd_values = []
cond_sat = []
num_sim = 50000
threshold = 0.05

multi_gbm_second_order_mmd_model.model.eval()
with torch.no_grad():
    for i in range(num_sim):
        sigma_1 = np.random.uniform(0.2, 0.8)
        sigma_2 = np.random.uniform(0.2, 0.8)
        sigma_3 = np.random.uniform(0.2, 0.8)
        corr = np.random.uniform(0.05, 0.95)

        distance = torch.maximum(
            multi_gbm_second_order_mmd_model.transform(torch.tensor(
                [sigma_1, sigma_1, sigma_2, sigma_2, sigma_3, sigma_3, corr, corr]
            ).unsqueeze(0).float()).squeeze(0).squeeze(0), torch.tensor(0.0)).cpu()
        
        mmd_values.append(distance)
        
        if distance < threshold:
            cond_sat.append(True)
        else:
            cond_sat.append(False)
            
counter_dict = Counter(cond_sat)
print(f'Number of Samples: {num_sim}')
print(f'Threshold: {threshold}')
print(f'Number of Distances Below Threshold: {counter_dict[True]}')

Number of Samples: 50000
Threshold: 0.05
Number of Distances Below Threshold: 49336


- #### Symmetry: $\sqrt{\mathcal{NN}_{2}^{\text{MGBM}} \left(\Theta_{1}, \Theta_{2} \right)} = \sqrt{\mathcal{NN}_{2}^{\text{MGBM}} \left(\Theta_{2}, \Theta_{1} \right)}$. 

In [24]:
threshold = 0.05
num_sim = 50000

is_sat = []

multi_gbm_second_order_mmd_model.model.eval()
with torch.no_grad():
    for i in range(num_sim):
        
        sigma_1_1 = np.random.uniform(0.2, 0.8)
        sigma_2_1 = np.random.uniform(0.2, 0.8)
        sigma_3_1 = np.random.uniform(0.2, 0.8)
        corr_1 = np.random.uniform(0.05, 0.95)
        
        sigma_1_2 = np.random.uniform(0.2, 0.8)
        sigma_2_2 = np.random.uniform(0.2, 0.8)
        sigma_3_2 = np.random.uniform(0.2, 0.8)
        corr_2 = np.random.uniform(0.05, 0.95)

        d1 = torch.maximum(
            multi_gbm_second_order_mmd_model.transform(torch.tensor(
                [sigma_1_1, sigma_1_2, sigma_2_1, sigma_2_2, sigma_3_1, sigma_3_2, corr_1, corr_2]
            ).unsqueeze(0).float()).squeeze(0).squeeze(0), torch.tensor(0.0)).cpu()
        
        d2 = torch.maximum(
            multi_gbm_second_order_mmd_model.transform(torch.tensor(
                [sigma_1_2, sigma_1_1, sigma_2_2, sigma_2_1, sigma_3_2, sigma_3_1, corr_2, corr_1]
            ).unsqueeze(0).float()).squeeze(0).squeeze(0), torch.tensor(0.0)).cpu()
                
        if abs(np.sqrt(d1)-np.sqrt(d2)) < threshold:
            is_sat.append(True)
        else:
            is_sat.append(False)

symm_counter_dict = Counter(is_sat)
print(f'Number of Samples: {num_sim}')
print(f'Threshold: {threshold}')
print(f'Number of Distances Below Threshold: {symm_counter_dict[True]}')

Number of Samples: 50000
Threshold: 0.05
Number of Distances Below Threshold: 38741


- #### Triangle Inequality: $\sqrt{\mathcal{NN}_{\mathcal{D}^{2}}^{\text{MGBM}} \left(\Theta_{1}, \Theta_{3} \right)} \leq \sqrt{\mathcal{NN}_{\mathcal{D}^{2}}^{\text{MGBM}} \left(\Theta_{1}, \Theta_{2} \right)} + \sqrt{\mathcal{NN}_{\mathcal{D}^{2}}^{\text{MGBM}} \left(\Theta_{2}, \Theta_{3} \right)}$. 

In [25]:
triag_inq_sat = []
num_sim = 50000

multi_gbm_second_order_mmd_model.model.eval()
with torch.no_grad():
    for i in range(num_sim):
        
        sigma_1_1 = np.random.uniform(0.2, 0.8)
        sigma_2_1 = np.random.uniform(0.2, 0.8)
        sigma_3_1 = np.random.uniform(0.2, 0.8)
        corr_1 = np.random.uniform(0.05, 0.95)
        
        sigma_1_2 = np.random.uniform(0.2, 0.8)
        sigma_2_2 = np.random.uniform(0.2, 0.8)
        sigma_3_2 = np.random.uniform(0.2, 0.8)
        corr_2 = np.random.uniform(0.05, 0.95)
        
        sigma_1_3 = np.random.uniform(0.2, 0.8)
        sigma_2_3 = np.random.uniform(0.2, 0.8)
        sigma_3_3 = np.random.uniform(0.2, 0.8)
        corr_3 = np.random.uniform(0.05, 0.95)

        d1 = torch.maximum(
            multi_gbm_second_order_mmd_model.transform(torch.tensor(
                [sigma_1_1, sigma_1_3, sigma_2_1, sigma_2_3, sigma_3_1, sigma_3_3, corr_1, corr_3]
            ).unsqueeze(0).float()).squeeze(0).squeeze(0), torch.tensor(0.0)).cpu()
        
        d2 = torch.maximum(
            multi_gbm_second_order_mmd_model.transform(torch.tensor(
                [sigma_1_1, sigma_1_2, sigma_2_1, sigma_2_2, sigma_3_1, sigma_3_2, corr_1, corr_2]
            ).unsqueeze(0).float()).squeeze(0).squeeze(0), torch.tensor(0.0)).cpu()
        
        d3 = torch.maximum(
            multi_gbm_second_order_mmd_model.transform(torch.tensor(
                [sigma_1_2, sigma_1_3, sigma_2_2, sigma_2_3, sigma_3_2, sigma_3_3, corr_2, corr_3]
            ).unsqueeze(0).float()).squeeze(0).squeeze(0), torch.tensor(0.0)).cpu()
        
        if np.sqrt(d1) <= (np.sqrt(d2)+np.sqrt(d3)):
            triag_inq_sat.append(True)
        else:
            triag_inq_sat.append(False)
            
triag_inq_counter_dict = Counter(triag_inq_sat)
print(f'Number of Samples: {num_sim}')
print(f'Number of Samples Satisfying the Triangle Inequality: {triag_inq_counter_dict[True]}')

Number of Samples: 50000
Number of Samples Satisfying the Triangle Inequality: 49937
